In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class DEM_Dataset(Dataset):
    def __init__(self, data_path):
        self.data = torch.load(data_path)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        data = self.data[index]
        return data

In [4]:
dataset = DEM_Dataset("/content/drive/My Drive/Data/DEM_slope_elevation_tensorv2.pt")

train_ratio = 0.7
test_ratio = 0.15
val_ratio = 0.15

total_samples = len(dataset)
train_size = int(train_ratio * total_samples)
test_size = int(test_ratio * total_samples)
val_size = total_samples - train_size - test_size


In [6]:
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

In [7]:
batch_size = 3

# Training DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Testing DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Validation DataLoader
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
import torch
import torch.nn as nn

class EncoderDecoder2D(nn.Module):
    def __init__(self):
        super(EncoderDecoder2D, self).__init__()

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=(15, 15), stride=4, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=(8, 8), stride=4, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=2, padding=1),
            nn.ReLU(inplace=True),

        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size = (4, 4), stride=2, padding = 1, output_padding = 0),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(64, 32, kernel_size=(8, 8), stride=4, padding=1, output_padding=0),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(32, 2, kernel_size=(15, 15), stride=4, padding=1, output_padding=0),
            nn.ReLU(inplace = True),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def dimensions(self, x):
       print("----------ENCODER-------------")
       print(f"Input shape: {x.shape}")
       x = self.encoder[0](x)
       print(f"1st Conv2D : {x.shape}")
       x = self.encoder[1](x)
       print(f"1st ReLU : {x.shape}")
       x = self.encoder[2](x)
       print(f"2nd Conv2D : {x.shape}")
       x = self.encoder[3](x)
       print(f"2nd ReLU: {x.shape}")
       x = self.encoder[4](x)
       print(f"3rd Conv2D : {x.shape}")
       x = self.encoder[5](x)
       print(f"3rd ReLU : {x.shape}")
       x = self.encoder[6](x)
       print(f"4th Conv2D : {x.shape}")
       x = self.encoder[7](x)
       print(f"4th ReLU : {x.shape}")
       x = self.encoder[8](x)
       print(f"5th Conv2D : {x.shape}")
       x = self.encoder[9](x)
       print(f"5th ReLU : {x.shape}")




       print("---------DECODER---------")
       x = self.decoder[0](x)
       print(f"1st Conv2D Transpose: {x.shape}")
       x = self.decoder[1](x)
       print(f"1st RelU: {x.shape}")
       x = self.decoder[2](x)
       print(f"2nd Conv2D Transpose: {x.shape}")
       x = self.decoder[3](x)
       print(f"2nd ReLU: {x.shape}")
       x = self.decoder[4](x)
       print(f"3rd Conv2D Transpose: {x.shape}")
       x = self.decoder[5](x)
       print(f"3rd ReLU: {x.shape}")
       x = self.decoder[6](x)
       print(f"4th Conv2D: {x.shape}")
       x = self.decoder[7](x)
       print(f"4th ReLU: {x.shape}")
       x = self.decoder[8](x)
       print(f"5th Conv2D: {x.shape}")
       x = self.decoder[9](x)
       print(f"5th ReLU: {x.shape}")

# Create an instance of the EncoderDecoder2D model
encoder_decoder_2d_model = EncoderDecoder2D()

# Assuming you have a batch of input data with shape [batch_size, 2, height, width]
batch_size = 1
height = 3601
width = 3601
dummy_input_batch = torch.rand(batch_size, 2, height, width)

# Pass the input batch through the model
#output_batch = encoder_decoder_2d_model(dummy_input_batch)
#print(output_batch.shape)  # This should print [batch_size, 2, height, width] (after the decoder upsampling operations).


In [7]:
encoder_decoder_2d_model.dimensions(dummy_input_batch)

----------ENCODER-------------
Input shape: torch.Size([1, 2, 3601, 3601])
1st Conv2D : torch.Size([1, 32, 898, 898])
1st ReLU : torch.Size([1, 32, 898, 898])
2nd Conv2D : torch.Size([1, 64, 224, 224])
2nd ReLU: torch.Size([1, 64, 224, 224])
3rd Conv2D : torch.Size([1, 128, 112, 112])
3rd ReLU : torch.Size([1, 128, 112, 112])
4th Conv2D : torch.Size([1, 256, 56, 56])
4th ReLU : torch.Size([1, 256, 56, 56])
5th Conv2D : torch.Size([1, 512, 28, 28])
5th ReLU : torch.Size([1, 512, 28, 28])
---------DECODER---------
1st Conv2D Transpose: torch.Size([1, 256, 56, 56])
1st RelU: torch.Size([1, 256, 56, 56])
2nd Conv2D Transpose: torch.Size([1, 128, 112, 112])
2nd ReLU: torch.Size([1, 128, 112, 112])
3rd Conv2D Transpose: torch.Size([1, 64, 224, 224])
3rd ReLU: torch.Size([1, 64, 224, 224])
4th Conv2D: torch.Size([1, 32, 898, 898])
4th ReLU: torch.Size([1, 32, 898, 898])
5th Conv2D: torch.Size([1, 2, 3601, 3601])
5th ReLU: torch.Size([1, 2, 3601, 3601])


In [10]:
batch_size = 3
data_loader = DataLoader(dataset, batch_size=3, shuffle=True)

# Create an instance of the EncoderDecoder2D model
encoder_decoder_2d_model = EncoderDecoder2D()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(encoder_decoder_2d_model.parameters(), lr=1e-5)

# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_decoder_2d_model.to(device)

for epoch in tqdm(range(num_epochs)):
    total_loss = 0.0

    for batch_data in train_dataloader:
        # Move data to the GPU if available
        batch_data = batch_data.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output_batch = encoder_decoder_2d_model(batch_data.float())

        # Calculate the loss
        loss = criterion(output_batch, batch_data.float())

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)

    val_loss = 0.0
    with torch.no_grad():
        for val_batch_data in val_dataloader:
            val_batch_data = val_batch_data.float().to(device)
            val_outputs = encoder_decoder_2d_model(val_batch_data)
            val_loss += criterion(val_outputs, val_batch_data).item()

    val_loss /= len(val_dataloader)

    #print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Val Loss: {val_loss}")

print("Training finished!")

Epoch 1/100, Loss: 1274518.2629, Val Loss: 2911186.173828125
Epoch 2/100, Loss: 1357719.5321, Val Loss: 2910875.7252604165
Epoch 3/100, Loss: 1262039.3315, Val Loss: 2909414.6295572915
Epoch 4/100, Loss: 1261976.3097, Val Loss: 2904861.6692708335
Epoch 5/100, Loss: 1260838.6950, Val Loss: 2887244.1064453125
Epoch 6/100, Loss: 1337845.5942, Val Loss: 2819548.9720052085
Epoch 7/100, Loss: 1278968.4418, Val Loss: 2571030.1806640625
Epoch 8/100, Loss: 985467.0213, Val Loss: 1960005.2962239583
Epoch 9/100, Loss: 729356.9497, Val Loss: 1411018.1796875
Epoch 10/100, Loss: 587001.4914, Val Loss: 1181414.3108723958
Epoch 11/100, Loss: 494129.7381, Val Loss: 1041610.7781575521
Epoch 12/100, Loss: 433233.2537, Val Loss: 941140.5094401041
Epoch 13/100, Loss: 393500.5775, Val Loss: 854893.8707682291
Epoch 14/100, Loss: 380567.3923, Val Loss: 790623.6995442709
Epoch 15/100, Loss: 335000.6402, Val Loss: 725282.2016601562
Epoch 16/100, Loss: 324440.5852, Val Loss: 673691.210530599
Epoch 17/100, Loss: 

In [13]:
encoder_decoder_2d_model.eval()  # Set the model to evaluation mode
test_loss = 0.0
with torch.no_grad():
    with tqdm(test_dataloader, desc='Testing', leave=False) as t:
        for test_batch_data in t:
            test_batch_data = test_batch_data.float().to(device)
            test_outputs = encoder_decoder_2d_model(test_batch_data)
            test_loss += criterion(test_outputs, test_batch_data).item()

test_loss /= len(test_dataloader)  # Average the test loss
print(f"Test Loss: {test_loss}")

Test Loss: 42251.949198404945
